# Character Network Analysis

Let's look at the network generated by the character tags on AO3. In order to use the data with Gephi to generate a social network graph, we need to put the data into the nodes/edges format.

In [2]:
#imports
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
import warnings; warnings.simplefilter('ignore')
import json
import pandas as pd

In [3]:
#read in data
character_df = pd.read_json('../ao3bot/character_network.json')
character_df.head()

,fandom,character,total_works,characters,relationships,ratings,warnings,categories,fandoms,freeforms
0,僕のヒーローアカデミア | Boku no Hero Academia | My Hero ...,Midoriya Izuku,105924,"{'Midoriya Izuku': '105924', 'Bakugou Katsuki'...","{'Bakugou Katsuki/Midoriya Izuku': '25771', 'M...","{'Teen And Up Audiences': '41011', 'General Au...","{'No Archive Warnings Apply': '47385', 'Creato...","{'M/M': '57991', 'Gen': '24432', 'F/M': '23647...",{'僕のヒーローアカデミア | Boku no Hero Academia | My Her...,"{'Fluff': '18652', 'Angst': '15311', 'Bakugou ..."
1,僕のヒーローアカデミア | Boku no Hero Academia | My Hero ...,Bakugou Katsuki,99662,"{'Bakugou Katsuki': '99662', 'Midoriya Izuku':...","{'Bakugou Katsuki/Midoriya Izuku': '25740', 'B...","{'Teen And Up Audiences': '38968', 'General Au...","{'No Archive Warnings Apply': '45628', 'Creato...","{'M/M': '62205', 'F/M': '23462', 'Gen': '16653...",{'僕のヒーローアカデミア | Boku no Hero Academia | My Her...,"{'Fluff': '18840', 'Bakugou Katsuki Swears A L..."
2,僕のヒーローアカデミア | Boku no Hero Academia | My Hero ...,Todoroki Shouto,60147,"{'Todoroki Shouto': '60147', 'Midoriya Izuku':...","{'Midoriya Izuku/Todoroki Shouto': '17548', 'B...","{'Teen And Up Audiences': '23674', 'General Au...","{'No Archive Warnings Apply': '27817', 'Creato...","{'M/M': '37338', 'F/M': '15150', 'Gen': '11340...",{'僕のヒーローアカデミア | Boku no Hero Academia | My Her...,"{'Fluff': '12201', 'Angst': '8991', 'Bakugou K..."
3,僕のヒーローアカデミア | Boku no Hero Academia | My Hero ...,Kirishima Eijirou,54460,"{'Kirishima Eijirou': '54460', 'Bakugou Katsuk...","{'Bakugou Katsuki/Kirishima Eijirou': '21943',...","{'Teen And Up Audiences': '21545', 'General Au...","{'No Archive Warnings Apply': '26077', 'Creato...","{'M/M': '36734', 'F/M': '13639', 'Gen': '7721'...",{'僕のヒーローアカデミア | Boku no Hero Academia | My Her...,"{'Fluff': '12161', 'Bakugou Katsuki Swears A L..."
4,僕のヒーローアカデミア | Boku no Hero Academia | My Hero ...,Aizawa Shouta | Eraserhead,52912,"{'Aizawa Shouta | Eraserhead': '52912', 'Midor...",{'Aizawa Shouta | Eraserhead/Yamada Hizashi | ...,"{'Teen And Up Audiences': '21409', 'General Au...","{'No Archive Warnings Apply': '22912', 'Creato...","{'M/M': '26120', 'Gen': '15076', 'F/M': '12639...",{'僕のヒーローアカデミア | Boku no Hero Academia | My Her...,{'Parental Aizawa Shouta | Eraserhead': '10405...


In [17]:
nodes = []
nodes_unformatted = []
ships = character_df["relationships"].tolist()
count = 1
for ship in ships:
    l = character_df.loc[character_df["relationships"] == ship]["relationships"]
    if len(l) > 0:
        for i in l:
            for j in i:
                if "&" not in j:
                    split = j.split("/")
                    for char in split:
                        if char not in nodes_unformatted:
                            nodes.append([count, char])
                            nodes_unformatted.append(char)
                            count+=1
print(len(nodes))

622


In [18]:
nodes_df = pd.DataFrame(nodes, columns=["Id","Label"])
print(nodes_df.head())
nodes_df.to_csv("nodes_and_edges/ship_nodes.csv", index = False)

   Id                       Label
0   1             Bakugou Katsuki
1   2              Midoriya Izuku
2   3             Todoroki Shouto
3   4           Kirishima Eijirou
4   5  Aizawa Shouta | Eraserhead


In [19]:
nodes_df.loc[nodes_df.Label == 'Tony Stark','Id'].tolist()[0]

226

In [25]:
edges = []
ships = character_df["relationships"].tolist()
for ship in ships:
    l = character_df.loc[character_df["relationships"] == ship]["relationships"]
    if len(l) > 0:
        for i in l:
            for j in i:
                if "&" not in j and "/" in j:
                    split = j.split("/")
                    target = nodes_df.loc[nodes_df.Label == split[0],'Id'].tolist()[0]
                    src = nodes_df.loc[nodes_df.Label == split[1],'Id'].tolist()[0]
                    edges.append([src, target, "Undirected"])


print(len(edges))

3973


In [26]:
edges_df = pd.DataFrame(edges, columns = ["Source", "Target", "Type"])
edges_df

,Source,Target,Type
0,2,1,Undirected
1,3,2,Undirected
2,4,1,Undirected
3,6,5,Undirected
4,7,2,Undirected
...,...,...,...
3968,616,618,Undirected
3969,313,616,Undirected
3970,616,619,Undirected
3971,617,618,Undirected


In [27]:
edges_df.to_csv("nodes_and_edges/ships_edges.csv", index = False)